In [35]:
#Declarations Of Numpy,Pandas and other libraries.
import numpy
import pandas as pd
import copy
import random
import math
import operator
import heapq

MAXEXAM = 10  # maximum exams at a time
TIME = 3  # time for an exam
SIZE = 10  # size of the population
DAYS = 5 #Number Of Days

In [36]:
class Population: #class Used To Store Entire Population fitness value and the index of the desired value.
    fitn = -1
    ind = -1

    def __init__(self):
        self.fitn = -1
        self.ind = -1


In [37]:
#Class Slot Represents the single examslot which contains course,teacher,
#timeslot (starting and ending) and the day of the exam.
class Slot:
    course = ''
    teacher = ''
    timeslot = [9, 9 + TIME]
    day = 1

    def __init__(self, *args):
        if len(args) > 0:
            self.course = args[0]
            self.teacher = args[1]


In [38]:
#Class Student which represents name,coursecode array which contains the course codes of the courses studied by the student
# and the day array which contains the day of exam of each student course.
class Student:
    name = ""
    coursecode = []
    coursetimeslot = []
    day = []

    def __init__(self, *args):
        if len(args) > 0:
            self.name = args[0]
            self.coursecode = args[1]
            self.day = args[2]
        else:
            self.name = ""
            self.coursecode = []
            self.coursetimeslot = []
            self.day = []


In [39]:
# Generates population based on courses teacher and population size. Return population at each function call.
def generate_pop(courses, teachers, popsize):
    no_of_courses = len(courses)
    population = []
    index = 0

    while index < popsize:
        examslot = [Slot() for i in range(no_of_courses)]
        
        #Hard Constraint Satisfied
        starttime = 9
        now = starttime
        endtime = 17
        for i in range(len(examslot)):
            examslot[i].course = courses["Code"][i]
            examslot[i].day = random.randint(1, DAYS) #Hard Constraint Satisfied
            now = random.randint(starttime, endtime - 3)
            examslot[i].timeslot = [now, now + TIME]
            examslot[i].teacher = teachers["Name"][random.randint(0, len(teachers) - 1)]

        # examslot.sort(key=operator.attrgetter('day'))

        population.append(examslot)
        index += 1

    # for i in range(len(examslot)):
    #     print(examslot[i].course, examslot[i].day, examslot[i].timeslot, examslot[i].teacher)

    return population

In [40]:
# Calculates fitness by applying some of the hard constraints.
def fitness(stud, examslot):
    fitness_count = 3

    # Set student course timeslot according to the courses he/she is studying.
    for i in range(0, len(stud)):
        for j in range(0, len(examslot)):
            for k in range(0, len(stud[i].coursecode)):
                if stud[i].coursecode[k] == examslot[j].course:
                    stud[i].coursetimeslot.append(examslot[j].timeslot)
                    stud[i].day.append(examslot[j].day)

    # for i in range(len(stud)):
    #     print(stud[i].coursetimeslot, stud[i].day, sep='\n')
    
    #Apply that no student has an exam at the same day and time.
    for i in range(0, len(stud)):
        for j in range(0, len(stud[i].coursetimeslot)):
            for k in range(0, len(stud[i].coursetimeslot)):
                if k != j:
                    if stud[i].day[j] == stud[i].day[k]:
                        if stud[i].coursetimeslot[j][0] == stud[i].coursetimeslot[k][0] \
                                or stud[i].coursetimeslot[k][0] < stud[i].coursetimeslot[j][0] \
                                < stud[i].coursetimeslot[k][1]:
                            fitness_count -= 1
                            break
            else:
                continue
            break
        else:
            continue
        break

    # print(fitness_count)

    # Apply that no teacher invigilates more then 1 exam at a single day and time.
    for i in range(0, len(examslot)):
        for j in range(0, len(examslot)):
            if i != j:
                if examslot[i].teacher == examslot[j].teacher and examslot[i].day == examslot[j].day:
                    if examslot[i].timeslot[0] == examslot[j].timeslot[0] or \
                            examslot[i].timeslot[0] < examslot[j].timeslot[0] < examslot[i].timeslot[1]:
                        fitness_count -= 1
                        break
        else:
            continue
        break

    # Check if Teacher Has Two Exams In a Row.
    for i in range(0, len(examslot)):
        for j in range(0, len(examslot)):
            if i != j:
                if examslot[i].teacher == examslot[j].teacher and examslot[i].day == examslot[j].day:
                    if examslot[i].timeslot[1] == examslot[j].timeslot[0]:
                        fitness_count -= 1
                        break
        else:
            continue
        break

    return fitness_count

In [41]:
def fitness_check(stud, examslot): #For Used By Soft Constraints
    fitness_count = 3

    for i in range(0, len(stud)):
        for j in range(0, len(stud[i].coursetimeslot)):
            for k in range(0, len(stud[i].coursetimeslot)):
                if k != j:
                    if stud[i].day[j] == stud[i].day[k]:
                        if stud[i].coursetimeslot[j][0] == stud[i].coursetimeslot[k][0] \
                                or stud[i].coursetimeslot[k][0] < stud[i].coursetimeslot[j][0] \
                                < stud[i].coursetimeslot[k][1]:
                            fitness_count -= 1
                            break
            else:
                continue
            break
        else:
            continue
        break

    # print(fitness_count)

    # Check Teacher Clash With Other Slots
    for i in range(0, len(examslot)):
        for j in range(0, len(examslot)):
            if i != j:
                if examslot[i].teacher == examslot[j].teacher and examslot[i].day == examslot[j].day:
                    if examslot[i].timeslot[0] == examslot[j].timeslot[0] or \
                            examslot[i].timeslot[0] < examslot[j].timeslot[0] < examslot[i].timeslot[1]:
                        fitness_count -= 1
                        break
        else:
            continue
        break

    # Check if Teacher Has Two Exams In a Row
    for i in range(0, len(examslot)):
        for j in range(0, len(examslot)):
            if i != j:
                if examslot[i].teacher == examslot[j].teacher and examslot[i].day == examslot[j].day:
                    if examslot[i].timeslot[1] == examslot[j].timeslot[0]:
                        fitness_count -= 1
                        break
        else:
            continue
        break

    return fitness_count

In [42]:
def roulette(fitness_value, pop): #Roulette Wheel Selection For Potential Chromosomes.
    max = sum([c.fitn for c in fitness_value])
    pick = random.uniform(0, max)
    current = 0
    for chromosome in range(len(pop)):
        current += fitness_value[chromosome].fitn
        if current > pick:
            return chromosome

In [43]:

def crossover(fit1, fit2, pop, no_of_courses): #Applying single point crossover between the two parents.
    child = [Slot() for i in range(no_of_courses)]

    single_point = random.randint(1, len(pop[fit1]) - 1)

    for i in range(len(pop[fit1])):
        if i < single_point:
            child[i] = pop[fit1][i]
        else:
            child[i] = pop[fit2][i]

    return child

In [44]:
def Check_Soft_Contraints1(stud, child):
    #Checking Soft Constraint 1 i.e if an exam has overlapped time on Friday at 1-2PM.
    #We Change The Timing To 2-5 and check if soft constraint can be satisfied or not.
    child1 = copy.deepcopy(child)

    ''' If Any Paper Has Timing Within 1-2PM On Friday '''
    for i in range(0, len(child1)):
        if child1[i].day % 5 == 0:
            if (13 <= child1[i].timeslot[0] < 14) or (13 < child1[i].timeslot[1] <= 14) or (
                    child1[i].timeslot[0] <= 13 and child1[i].timeslot[1] > 14) or (
                    child1[i].timeslot[0] < 13 and child1[i].timeslot[1] >= 14):
                child1[i].timeslot[0] = 14
                child1[i].timeslot[1] = 17
                for j in range(0, len(stud)):
                    for k in range(0, len(stud[j].coursecode)):
                        if stud[j].day[k] % 5 == 0 and stud[j].coursecode[k] == child1[i].course:
                            stud[j].coursetimeslot[k][0] = 14
                            stud[j].coursetimeslot[k][1] = 17

    value = fitness_check(stud, child1)

    if value != 3:
        return child1
    else:
        return None


def Check_Soft_Contraints2(stud, child):
    #Checking Soft Constraint 2 i.e a student shall not give 2 exams concurrently.
    child1 = copy.deepcopy(child)

    ''' Check if student has consecutive exams '''
    for i in range(0, len(stud)):
        for j in range(0, len(stud[i].day)):
            for k in range(0, len(stud[i].day)):
                if j != k:
                    if stud[i].day[j] == stud[i].day[k]:
                        if stud[i].coursetimeslot[j][1] == stud[i].coursetimeslot[k][0]:
                            if stud[i].coursetimeslot[k][1] + 1 <= 17:
                                stud[i].coursetimeslot[k][0] += 1
                                stud[i].coursetimeslot[k][1] += 1
                                for l in range(0, len(child1)):
                                    if stud[i].coursecode[k] == child1[l].course:
                                        child1[l].timeslot[0] += 1
                                        child1[l].timeslot[1] += 1

    value = fitness_check(stud, child1)

    if value != 3:
        return child1
    else:
        return None


def Check_Soft_Contraints3(stud, child):
    
    #Checking Soft Constraint 3 i.e a student shall give MG exam before his CS Exam.
    for i in range(len(stud)):
        for j in range(len(stud[i].coursecode)):
            for k in range(len(stud[i].coursecode)):
                if j != k:
                    if stud[i].coursecode[j][0:2] == "MG" and stud[i].coursecode[k][0:2] == "CS":
                        if stud[i].day[j] > stud[i].day[k]:
                            stud[i].day[j], stud[i].day[k] = stud[i].day[k], stud[i].day[j]
                            stud[i].coursetimeslot[j], stud[i].coursetimeslot[k] = stud[i].coursetimeslot[k], \
                                                                                   stud[i].coursetimeslot[j]
                            for l in range(len(child)):
                                if stud[i].coursecode[j] == child[l].course:
                                    child[l].timeslot = stud[i].coursetimeslot[j]
                                    child[l].day = stud[i].day[j]
                                if stud[i].coursecode[k] == child[l].course:
                                    child[l].timeslot = stud[i].coursetimeslot[k]
                                    child[l].day = stud[i].day[k]
                        elif stud[i].day[j] == stud[i].day[k]:
                            if stud[i].coursetimeslot[j][0] > stud[i].coursetimeslot[k][1]:
                                stud[i].day[j], stud[i].day[k] = stud[i].day[k], stud[i].day[j]
                                stud[i].coursetimeslot[j], stud[i].coursetimeslot[k] = stud[i].coursetimeslot[k], \
                                                                                       stud[i].coursetimeslot[j]
                                for l in range(len(child)):
                                    if stud[i].coursecode[j] == child[l].course:
                                        child[l].timeslot = stud[i].coursetimeslot[j]
                                        child[l].day = stud[i].day[j]
                                    if stud[i].coursecode[k] == child[l].course:
                                        child[l].timeslot = stud[i].coursetimeslot[k]
                                        child[l].day = stud[i].day[k]

    if fitness_check(stud, child):
        return child
    else:
        return None


def Check_Soft_Contraints4(stud, child):
    return None


In [45]:
def main():
    solution = None
    col_stud = ["Name"]
    col_course = ["Code", "Name"]
    col_teach = ["Name"]
    col_studcourse = ["ID", "Student Name", "Course Code"]
    students = pd.read_csv('studentNames.csv', usecols=col_stud)
    students.drop_duplicates(subset="Name", inplace=True)
    students.reset_index(drop=True, inplace=True)

    courses = pd.read_csv('courses.csv', usecols=col_course)
    courses.drop_duplicates(subset="Code", inplace=True)
    courses.reset_index(drop=True, inplace=True)

    teachers = pd.read_csv('teachers.csv', usecols=col_teach)
    teachers.drop_duplicates(subset="Name", inplace=True)
    teachers.reset_index(drop=True, inplace=True)

    studentcourses = pd.read_csv('studentCourse.csv', usecols=col_studcourse)

    # generating population
    pop = generate_pop(courses, teachers, SIZE)

    fitnessval = []

    # Calculating fitness value (lower is better)
    while True:
        fitnessval.clear()
        stud = [[Student() for i in range(len(students))] for j in range(len(pop))]

        for l in range(0, len(pop)):

            # Set student name
            for i in range(len(students)):
                stud[l][i].name = students["Name"][i]

            # Set student course codes
            for i in range(0, len(stud[l])):
                for j in range(0, len(studentcourses)):
                    if stud[l][i].name == studentcourses["Student Name"][j]:
                        stud[l][i].coursecode.append(studentcourses["Course Code"][j])

            p1 = Population()
            p1.fitn = fitness(stud[l], pop[l])
            p1.ind = l

            fitnessval.append(p1)

        fit1 = roulette(fitnessval, pop)

        fit2 = roulette(fitnessval, pop)

        no_of_courses = len(courses)
        child = crossover(fit1, fit2, pop, no_of_courses)

        # for i in range(len(child)):
        #     print(child[i].day, child[i].timeslot, child[i].course, child[i].teacher)

        pop.append(child)

        temp = [Student() for i in range(len(students))]
        for i in range(len(students)):
            temp[i].name = students["Name"][i]

        # Set student course codes
        for i in range(0, len(temp)):
            for j in range(0, len(studentcourses)):
                if temp[i].name == studentcourses["Student Name"][j]:
                    temp[i].coursecode.append(studentcourses["Course Code"][j])

        child_fit = fitness(temp, child)

        # for i in range(len(temp)):
        #     print(temp[i].day)

        print(fitnessval[fit1].fitn, fitnessval[fit2].fitn, child_fit)

        if child_fit == 3:

            print("Solution found!\nBefore:")
            deepchild1 = copy.deepcopy(child)
            deepchild2 = copy.deepcopy(child)
            deepchild3 = copy.deepcopy(child)
            temp1 = copy.deepcopy(temp)
            temp2 = copy.deepcopy(temp)
            temp3 = copy.deepcopy(temp)

            child.sort(key=operator.attrgetter('day'))
            for i in range(len(child)):
                print(child[i].course, child[i].day, child[i].timeslot, child[i].teacher)

            for i in range(len(temp)):
                print(temp[i].name, temp[i].coursecode, temp[i].day, temp[i].coursetimeslot,sep="\n")
            print("After:")

            return_flag1 = Check_Soft_Contraints1(temp1, deepchild1)
            if return_flag1 is not None:
                print("Soft Constraint 1 Satisfied")
                return_flag1.sort(key=operator.attrgetter('day'))
                for i in range(len(return_flag1)):
                    print(return_flag1[i].course, return_flag1[i].day, return_flag1[i].timeslot,
                          return_flag1[i].teacher)

                for i in range(0, len(temp1)):
                    print(temp1[i].name, temp1[i].day, temp1[i].coursecode, temp1[i].coursetimeslot, sep="\n")

            return_flag2 = Check_Soft_Contraints2(temp2, deepchild2)
            if return_flag2 is not None:
                print("Soft Constraint 2 Satisfied")
                return_flag2.sort(key=operator.attrgetter('day'))
                for i in range(len(return_flag2)):
                    print(return_flag2[i].course, return_flag2[i].day, return_flag2[i].timeslot,
                          return_flag2[i].teacher)

                for i in range(0, len(temp2)):
                    print(temp2[i].name, temp2[i].day, temp2[i].coursecode, temp2[i].coursetimeslot, sep="\n")

            return_flag3 = Check_Soft_Contraints3(temp3, deepchild3)
            if return_flag3 is not None:
                print("Soft Constraint 3 Satisfied")
                return_flag3.sort(key=operator.attrgetter('day'))
                for i in range(len(return_flag3)):
                    print(return_flag3[i].course, return_flag3[i].day, return_flag3[i].timeslot,
                          return_flag3[i].teacher)

                for i in range(0, len(temp3)):
                    print(temp3[i].name, temp3[i].day, temp3[i].coursecode, temp3[i].coursetimeslot, sep="\n")

            if return_flag1 is None and return_flag2 is None and return_flag3 is None:
                print("Soft Constraint Not Satisfied")
                child.sort(key=operator.attrgetter('day'))
                for i in range(len(child)):
                    print(child[i].course, child[i].day, child[i].timeslot, child[i].teacher)

                for i in range(0, len(temp)):
                    print(temp[i].name, temp[i].day, temp[i].coursecode, temp[i].coursetimeslot, sep="\n")

            break


if __name__ == '__main__':
    main()


2 2 2
1 2 1
2 2 2
2 2 2
2 2 1
2 2 2
2 1 2
2 2 2
2 2 2
2 2 2
2 2 2
2 2 2
2 2 2
2 2 2
2 2 2
2 2 2
2 2 2
2 2 2
2 2 2
2 2 2
1 2 2
1 2 1
2 2 2
2 2 1
2 2 2
2 2 2
2 2 1
2 2 2
2 1 2
2 2 2
1 2 1
2 2 2
2 2 2
2 2 2
2 2 2
2 2 2
2 2 2
2 2 2
2 2 2
2 2 2
2 2 2
2 2 2
2 2 1
2 2 2
2 2 2
2 2 2
2 2 2
2 2 1
2 2 2
2 2 2
2 2 1
2 2 2
2 2 2
2 2 1
2 2 1
2 2 2
2 2 1
1 1 2
2 2 2
2 2 2
2 2 2
2 2 2
1 2 1
2 2 1
2 2 2
2 2 2
2 2 1
2 2 1
2 2 2
2 2 3
Solution found!
Before:
CS211 1 [10, 13] Maimoona Rassol
SE110 1 [9, 12] Aqeel Shahzad
CS220 1 [14, 17] Asif Naeem
CS218 1 [9, 12] Shoaib Mehboob
SS113 1 [11, 14] Muhammad Asim
MG220 1 [13, 16] Adnan Tariq
AI2011 2 [9, 12] Zeeshan Qaiser
DS3011 2 [13, 16] Muhammad Asim
MG223 2 [12, 15] Zainab Moin
SS118 2 [9, 12] Faisal Cheema
CS118 3 [11, 14] Hammad Majeed
CS302 3 [12, 15] Javaria Imtiaz
EE229 3 [13, 16] Arshad Islam
SS111 3 [14, 17] Farah Naz
SS152 3 [13, 16] Umair Arshad
MT205 3 [10, 13] Usman Ashraf
CS217 4 [12, 15] Khadija Farooq
CS219 4 [10, 13] Subhan Ullah
CY2012 4 